In [34]:
import os
import pandas as pd
import numpy as np
import pyreadstat as prs

import config as c

In [35]:
print(c.db2018)
for f in sorted(os.listdir(c.db2018)):
    print(f)

../GEOStat/2018-DB_eng/2018 DB
123.sav
conspurch_03.sav
deadmen.sav
familysize.sav
sysschedule.sav
tblavgprices.sav
tblconsumption.sav
tblexpenditures.sav
tblincomes.sav
tblshinda01.sav
tblshinda01_animals.sav
tblshinda01_comfort.sav
tblshinda01_cultures.sav
tblshinda01_lands.sav
tblshinda01_subjects.sav
tblshinda02.sav
tblshinda03_1.sav
tblshinda04.sav
tblshinda04_1.sav


In [39]:
stataDirs = [c.db2018, 
             c.db2017, 
             c.db2016, 
             c.db2015, 
             c.db2014, 
             c.db2013, 
             c.db2012, 
             c.db2011, 
             c.db2010, 
             c.db2009]

for stataDir in stataDirs:
    print('\n')
    print(stataDir)
    cols = 0
    savs = 0
    for f in os.listdir(stataDir):
        if 'sysschedule' not in f:
            continue
        filePath = os.path.join(stataDir, f)
        print(f)
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print(df.head())
        print('count: {} -- {}'.format(len(df.RegNo.unique()), np.sort(df.RegNo.unique()).astype(int)))
        print('number_rows: {}'.format(meta.number_rows))
        print('column_names, n = {}'.format(len(meta.column_names), meta.column_names))
        print('column_labels, n = {}\n'.format(len(meta.column_labels), meta.column_labels))
        colLabels = meta.column_labels
        for l in colLabels:
            print(l)
    break
    print('total savs = {}'.format(savs))
    print('total cols = {}'.format(cols))



../GEOStat/2018-DB_eng/2018 DB
sysschedule.sav
      MonthYear  QuartNo       UID   DiaryID  RegNo  UrbanOrRural     Weights
0  1.373414e+10     87.0  321260.0  107549.0    0.0           1.0  135.234085
1  1.373414e+10     87.0  321261.0  107550.0    0.0           1.0  135.234085
2  1.373414e+10     87.0  321262.0  107551.0    0.0           1.0  135.234085
3  1.373414e+10     87.0  321263.0  107552.0    0.0           1.0  135.234085
4  1.373414e+10     87.0  321264.0  107553.0    0.0           1.0  135.234085
count: 10 -- [ 0  1  2  3  5  7  8  9 10 11]
number_rows: 11056
column_names, n = 7
column_labels, n = 7

Month and Year of Interview
Quarter ID
Unique ID of the household in the quarter
Unique ID of the household during the whole period
Region
Urban Or Rural
Sampling weights of the household


In [83]:
for stataDir in stataDirs:
    n=1
    print('\n{}'.format(stataDir))
    for f in os.listdir(stataDir):
        if 'tblshinda01' not in f:
            continue
        filePath = os.path.join(stataDir, f)
        print('#{} {:>5s} {}'.format(n, '', f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print(df.info())
        #print('\n')
        print('{}'.format('-'*40))
        for l in meta.column_labels:
            print(l)
        
        print('{}\n'.format('-'*80))
        n+=1
        
    break


../GEOStat/2018-DB_eng/2018 DB
#1       tblshinda01_lands.sav
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12579 entries, 0 to 12578
Data columns (total 5 columns):
UID            12579 non-null float64
LandNo         12579 non-null float64
SHeqtars       12579 non-null float64
PrivateForm    12579 non-null float64
Distance       12579 non-null float64
dtypes: float64(5)
memory usage: 491.4 KB
None
----------------------------------------
Unique ID of the household in the quarter
Number of the land plot
Land plot area (in hectares)
Form of ownership of land plot
Distance from the dwelling to the land plot (in kilometres)
--------------------------------------------------------------------------------

#2       tblshinda01_subjects.sav
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82123 entries, 0 to 82122
Data columns (total 5 columns):
UID                     82123 non-null float64
Subject                 82123 non-null float64
Quantity                82123 non-null float64
